<a href="https://colab.research.google.com/github/monolixd/Question-Answering-Workshop/blob/main/Question_Answering_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## โหลดโมเดล Pre-trained

In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# โหลด Tokenizer และโมเดล
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## ป้อนคำถามและบริบท

In [22]:
question = "who is the president of United States"
context = "Joe Biden, the 46th president of the United States, assumed office on January 20, 2021, following his victory in the 2020 presidential election. Representing the Democratic Party, Biden has prioritized policies aimed at uniting a divided nation, addressing climate change, and strengthening the economy. With decades of political experience, including serving as vice president under Barack Obama from 2009 to 2017, Biden's leadership emphasizes diplomacy, inclusivity, and rebuilding alliances on the global stage. His presidency has also focused on combating the COVID-19 pandemic and promoting initiatives for infrastructure development and social equity."

## แปลงข้อความเป็น Token IDs

In [23]:
inputs = tokenizer(question, context, return_tensors="pt", truncation=True)

## ใช้โมเดลทำนายคำตอบ

In [24]:
outputs = model(**inputs)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# หาตำแหน่งเริ่มต้นและสิ้นสุดของคำตอบ
start_index = start_scores.argmax()
end_index = end_scores.argmax()

## แปลงตำแหน่ง Token IDs กลับเป็นข้อความ

In [25]:
answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1])
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: who is the president of United States
Answer: joe biden


## การประเมินผล


In [26]:
true_answer = "Joe Biden"
em = int(answer.strip().lower() == true_answer.lower())
print(f"Exact Match: {em}")

Exact Match: 1


In [28]:
# แปลงคำตอบให้เป็น tokenized words
true_tokens = true_answer.strip().lower().split()
pred_tokens = answer.strip().lower().split()

# หาคำที่ตรงกัน
common = set(true_tokens) & set(pred_tokens)
precision = len(common) / len(pred_tokens) if pred_tokens else 0
recall = len(common) / len(true_tokens) if true_tokens else 0
f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

print(f"True Tokens: {true_tokens}")
print(f"Predicted Tokens: {pred_tokens}")
print(f"F1-Score: {f1_score:.2f}")

True Tokens: ['joe', 'biden']
Predicted Tokens: ['joe', 'biden']
F1-Score: 1.00


## Test